In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import numpy as np

In [3]:
# Device setup (switch to "cpu" if there's a memory issue)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [4]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
# Load dataset
dataset = datasets.ImageFolder(r'C:\Users\deepa\Downloads\Major2025\Disease-Tomato\PlantVillage', transform=transform)


In [6]:
# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [8]:
# Data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0)

In [9]:
# Model setup
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(dataset.classes))  # Update final layer for custom classes
model = model.to(device)

c:\Users\deepa\Downloads\2025\Disease-Tomato\.conda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\deepa\Downloads\2025\Disease-Tomato\.conda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
# Training loop
num_epochs = 10
print("Starting training...")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    print(f"Epoch {epoch + 1}/{num_epochs}")
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        # Print progress within each batch
        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch + 1} completed, Loss: {epoch_loss:.4f}")

Starting training...
Epoch 1/10
Batch 0/16, Loss: 1.2476
Batch 10/16, Loss: 0.0090
Epoch 1 completed, Loss: 0.3677
Epoch 2/10
Batch 0/16, Loss: 0.0192
Batch 10/16, Loss: 0.2577
Epoch 2 completed, Loss: 0.2258
Epoch 3/10
Batch 0/16, Loss: 0.3178
Batch 10/16, Loss: 0.2159
Epoch 3 completed, Loss: 0.1189
Epoch 4/10
Batch 0/16, Loss: 0.4150
Batch 10/16, Loss: 0.2435
Epoch 4 completed, Loss: 0.1268
Epoch 5/10
Batch 0/16, Loss: 0.0063
Batch 10/16, Loss: 0.0092
Epoch 5 completed, Loss: 0.0801
Epoch 6/10
Batch 0/16, Loss: 0.1933
Batch 10/16, Loss: 0.3047
Epoch 6 completed, Loss: 0.1234
Epoch 7/10
Batch 0/16, Loss: 0.0465
Batch 10/16, Loss: 0.0129
Epoch 7 completed, Loss: 0.0673
Epoch 8/10
Batch 0/16, Loss: 0.0042
Batch 10/16, Loss: 0.0011
Epoch 8 completed, Loss: 0.0274
Epoch 9/10
Batch 0/16, Loss: 0.0232
Batch 10/16, Loss: 0.0016
Epoch 9 completed, Loss: 0.1030
Epoch 10/10
Batch 0/16, Loss: 0.0694
Batch 10/16, Loss: 0.0226
Epoch 10 completed, Loss: 0.0212


In [15]:
from PIL import Image  # Import the Image class from PIL

def predict_image(image_path):
    image = Image.open(image_path)  # Open the image
    image = transform(image).unsqueeze(0).to(device)  # Apply transformations
    model.eval()
    with torch.no_grad():
        output = model(image)  # Get the model output
        _, predicted = torch.max(output, 1)  # Get the predicted class
    return dataset.classes[predicted.item()]  # Return the predicted class label

# Example usage with proper file path formatting
print(predict_image("Mosaic.jpg"))  # Using a raw string for the file path

Tomato_Bacterial_spot


In [16]:
# Save the trained model
torch.save(model.state_dict(), 'crop_disease_detection.pth')
print("Model saved as crop_disease_detection.pth")


Model saved as crop_disease_detection.pth
